In [10]:
import pandas as pd

# Load the datasets
time_distance = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance.csv')
zips_geocoded = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/zips_geocoded.csv')
walmart_geocoded = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/walmart_geocoded.csv')

# Merging to get RUCA and state for each ZIP code
time_distance = time_distance.merge(zips_geocoded[['zip', 'ruca', 'state_name']], left_on='zip', right_on='zip')

In [11]:
import random

# Sample size
sample_size = 4139

stratified_sample = time_distance.groupby('ruca', group_keys=False).apply(lambda x: x.sample(min(len(x), sample_size)))

# Selecting ZIP-Store combinations
sample_data = []

for index, row in stratified_sample.iterrows():
    stores = str(row['walmarts_pot_60']).split(' ')
    store = random.choice(stores)  # Randomly pick one store
    if store.isdigit():
        sample_data.append({'origin': row['zip'], 'destination': int(store), 'ruca': row['ruca']})

# Convert sample data to DataFrame
sample_df = pd.DataFrame(sample_data)

In [12]:
import pandas as pd
from geopy.distance import great_circle

# Define the function to get coordinates
def get_coords(store_number, zip_code):
    store_row = walmart_geocoded[walmart_geocoded['Store #'] == store_number]
    zip_row = zips_geocoded[zips_geocoded['zip'] == zip_code]
    if not store_row.empty and not zip_row.empty:
        store_coords = (store_row['latitude'].iloc[0], store_row['longitude'].iloc[0])
        zip_coords = (zip_row['latitude'].iloc[0], zip_row['longitude'].iloc[0])
        return zip_coords, store_coords
    return None, None

# Assuming sample_df is already prepared with 'destination' and 'origin' columns
sample_df['origin_lat'], sample_df['origin_lng'] = zip(*sample_df.apply(lambda x: get_coords(x['destination'], x['origin'])[0], axis=1))
sample_df['destination_lat'], sample_df['destination_lng'] = zip(*sample_df.apply(lambda x: get_coords(x['destination'], x['origin'])[1], axis=1))

# Calculate the great circle distance
sample_df['great_circle_distance'] = sample_df.apply(lambda x: great_circle((x['origin_lat'], x['origin_lng']), (x['destination_lat'], x['destination_lng'])).miles, axis=1)

# Final DataFrame with required columns
final_sample = sample_df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng', 'ruca', 'great_circle_distance']]

In [13]:
final_sample.to_csv('//Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv', index=False)

In [14]:
# Assuming final_sample is your DataFrame
final_sample['origin'] = final_sample['origin_lat'].astype(str) + ',' + final_sample['origin_lng'].astype(str)
final_sample['destination'] = final_sample['destination_lat'].astype(str) + ',' + final_sample['destination_lng'].astype(str)

# Drop the individual latitude and longitude columns
final_sample = final_sample.drop(['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng'], axis=1)

# Save the final DataFrame to the same CSV file
final_sample.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv', index=False)

/var/folders/_6/p9nq2tdd6418xp1vt5dxwqhm0000gn/T/ipykernel_84794/3630749054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_sample['origin'] = final_sample['origin_lat'].astype(str) + ',' + final_sample['origin_lng'].astype(str)


In [16]:
import pandas as pd

# Load the sample data
time_distance_sample = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv')

# Remove any duplicate rows (considering the entire row)
time_distance_sample = time_distance_sample.drop_duplicates()

# Check if the DataFrame has more rows than needed
num_rows_to_remove = len(time_distance_sample) - 4193

if num_rows_to_remove > 0:
    # Randomly delete rows until we have 4193 rows
    time_distance_sample = time_distance_sample.sample(n=4193)

# Save the updated DataFrame back to the CSV file
time_distance_sample.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv', index=False)

In [17]:
# Reordering the columns of the DataFrame
reordered_sample = time_distance_sample[['origin', 'destination', 'ruca', 'great_circle_distance']]

# Save the reordered DataFrame back to the CSV file
reordered_sample.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv', index=False)